<a href="https://colab.research.google.com/github/PittPrat/ResnetSymmetry/blob/master/DeepLearning_ImageComposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy.fftpack import fft2, fftshift
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')

# Set dataset path (All images in one folder)
dataset_path = "/content/drive/MyDrive/symmetry_dataset/"
image_paths = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith(".jpg")]

In [ ]:
# Preprocessing Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Load and preprocess images
images = np.array([preprocess_image(img) for img in image_paths])
print(f"Loaded {len(images)} images.")

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Unable to read image {image_path}")
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure correct color format
    image = cv2.resize(image, (224, 224))
    image = image.astype("float32") / 255.0
    return np.expand_dims(image, axis=0)

def compute_fourier_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Warning: Unable to process Fourier features for {image_path}")
        return None
    f_transform = np.fft.fft2(image)
    f_shift = np.fft.fftshift(f_transform)
    magnitude_spectrum = np.log1p(np.abs(f_shift))
    magnitude_spectrum_resized = cv2.resize(magnitude_spectrum, (224, 224))
    return magnitude_spectrum_resized.flatten()

In [ ]:
# Save Fourier Features for Future Use
fourier_features_path = "/content/drive/MyDrive/Output/fourier_features.pkl"
features_flattened = fourier_features.reshape(fourier_features.shape[0], -1)
with open(fourier_features_path, "wb") as f:
    pickle.dump(features_flattened, f)
print(f"Fourier features saved at: {fourier_features_path}")


In [ ]:
# Load saved fourier features
import pickle
fourier_features_path = "/content/drive/MyDrive/Output/fourier_features.pkl"
# Load Fourier Features (if needed)
def load_fourier_features(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

fourier_features = load_fourier_features(fourier_features_path)
print("Loaded Fourier features shape:", fourier_features.shape)

In [ ]:

features_flattened = fourier_features.reshape(fourier_features.shape[0], -1)

# Apply K-Means Clustering
num_clusters = 2  # Symmetric & Asymmetric
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans_labels = kmeans.fit_predict(features_flattened)

# Visualizing Clusters with PCA
pca = PCA(n_components=2)
pca_features = pca.fit_transform(features_flattened)
plt.scatter(pca_features[:, 0], pca_features[:, 1], c=kmeans_labels, cmap="viridis")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.title("Image Clusters based on Fourier Features")
plt.colorbar()
plt.show()

# Create labeled directories
os.makedirs("/content/drive/MyDrive/Output/Symmetric", exist_ok=True)
os.makedirs("/content/drive/MyDrive/Output/Asymmetric", exist_ok=True)

# Move images into respective clusters
for img_path, label in zip(image_paths, kmeans_labels):
    label_folder = "Symmetric" if label == 0 else "Asymmetric"
    os.rename(img_path, f"/content/drive/MyDrive/Output/{label_folder}/{os.path.basename(img_path)}")

# Load Auto-Labeled Data
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_data = datagen.flow_from_directory(
    "/content/drive/MyDrive/Output/",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)
val_data = datagen.flow_from_directory(
    "/content/drive/MyDrive/Output/",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"
)

# Load Pretrained ResNet50
base_model = ResNet50(weights="imagenet", include_top=False, pooling="avg", input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Define Model with Custom Classification Head
model = Sequential([
    base_model,
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")  # Binary classification
])

# Compile Model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    verbose=1
)

# Evaluate Model
loss, accuracy = model.evaluate(val_data)
print(f"Validation Accuracy: {accuracy:.2f}")

# Plot Training Results
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Model Accuracy")

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Model Loss")

plt.show()

# Save Model for Download
model_save_path = "/content/drive/MyDrive/Output/symmetry_classification_model.h5"
model.save(model_save_path)
print(f"Model saved at: {model_save_path}")


In [ ]:

# Predict on New Image
def predict_image(image_path):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    label = "Symmetric" if prediction >= 0.5 else "Asymmetric"
    print(f"Prediction for {image_path}: {label}")

# Example Usage
sample_image = "/content/drive/MyDrive/Test Resnet/TestOne.jpg"
predict_image(sample_image)


In [ ]:
# Define Test Data Path
test_data_path = "/content/drive/MyDrive/Test data/"

# Create an ImageDataGenerator for test set (No Augmentation, Just Rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=False  # No shuffling for test evaluation
)

# Print Number of Test Images
print(f"Loaded {test_data.samples} test images.")


In [ ]:
# Evaluate model on test dataset
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Final Test Accuracy: {test_accuracy:.2f}")



In [ ]:
def load_test_images(test_data_path):
    test_images = []
    test_labels = []
    file_names = [f for f in os.listdir(test_data_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for file_name in file_names:
        image_path = os.path.join(test_data_path, file_name)
        if not os.path.isfile(image_path):
            continue

        img_processed = preprocess_image(image_path)
        fourier_feature = compute_fourier_features(image_path)

        if img_processed is not None and fourier_feature is not None:
            test_images.append((img_processed, fourier_feature.reshape(1, 50176)))
            test_labels.append(0)  # Dummy label

    return test_images, test_labels


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

test_images, test_labels = load_test_images(test_data_path)

# Convert test data to arrays
test_x_images = np.vstack([img[0] for img in test_images]).astype(np.float32)
test_x_fourier = np.vstack([img[1] for img in test_images]).astype(np.float32)
test_x_fourier = test_x_fourier.reshape(len(test_x_fourier), 50176)  # Ensure correct shape
test_y = np.array(test_labels)

# Make predictions
predictions = model.predict([test_x_images, test_x_fourier])
predicted_classes = (predictions >= 0.5).astype(int).flatten()

# Compute metrics
accuracy = accuracy_score(test_y, predicted_classes)
precision = precision_score(test_y, predicted_classes, zero_division=1)
recall = recall_score(test_y, predicted_classes, zero_division=1)
f1 = f1_score(test_y, predicted_classes, zero_division=1)
conf_matrix = confusion_matrix(test_y, predicted_classes)
report = classification_report(test_y, predicted_classes)

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Classification Report:")
print(report)

# Plot confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

